In [1]:
import xarray as xr
import numpy as np
from scipy.io import loadmat
from scipy import interpolate, constants
from scipy import signal
import matplotlib as mpl
import matplotlib.pyplot as plt
import seastar as ss
import seastar.oscar
import seastar.retrieval
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import re
import warnings
import os
import platform
import datetime

In [5]:
# Data load and pre-processing
version = 'v20221104'
file_paths = ss.utils.readers._set_file_paths()
oscar_path = os.path.join(file_paths['oscar_path'], version, 'matlab-pre-processed','')
#oscar_path = "D:\\data\\SEASTAR\\SEASTARex\\Data\\Metasensing\\OSCAR\\May22_full_precompute_old\\"
#oscar_path = "D:\\data\\SEASTAR\\SEASTARex\\Data\\Metasensing\\OSCAR\\FINAL\\precompute\\20222205\\"
#oscar_path = "D:\\data\\SEASTAR\\SEASTARex\\Data\\Metasensing\\OSCAR\\FINAL\\delivered\\20222205\\"
#oscar_path = "D:\\data\\oscar\\v20221104\\matlab-pre-processed\\"

file_time_triplets = ss.utils.tools.find_file_triplets(oscar_path)

print('OSCAR L1 processing - version ' + version)
print('-------------------')
print('File path :', oscar_path)
print('-------------------')
for i in range(len(file_time_triplets)):
    print('File index :', i, ',', 'Aquisition time :', file_time_triplets[i][0], ',', 'File list indices :', file_time_triplets[i][1])

# Match tracks in the star pattern from track names and times in the DAR_v6.0 to file aquisition times
star_pattern_tracks={
    'Track_L1' : 0,
    'Track_1a' : 1,
    'Track_2a' : 2,
    'Track_1b' : 3,
    'Track_11' : 4,
    'Track_12' : 5,
    'Track_13' : 6,
    'Track_14' : 7,
    'Track_15' : 8,
    'Track_16' : 9,
    'Track_17' : 10,
    'Track_18' : 11}
# L1a variables to be passed to L1b dataset
vars_to_keep = [
        'LatImage',
        'LonImage',
        'IncidenceAngleImage',
        'SquintImage',
        ]
# Compute L1C - Sigma0 + RVL
ds_L1b = dict()

# Loop through star pattern tracks
for track in star_pattern_tracks.keys():
#for track in ['Track_15']:
    ds_ml = dict()
    file_index = star_pattern_tracks[track]
    ds_L1a = ss.utils.readers.load_OSCAR_data(oscar_path, file_time_triplets[file_index][1])
    ds_L1a_dict_index = list(ds_L1a.keys())
    antenna_ident = ss.utils.tools.identify_antenna_location_from_filename(
                        oscar_path,
                        file_time_triplets[file_index][1])
    print('Processing file index',file_index)
    
    # L1b processing - loop through beams in L1a dataset dict
    #--------------------------------------------------------
    for dict_index in ds_L1a_dict_index:
        ds_L1a[dict_index] = ss.oscar.level1.replace_dummy_values(
                                  ds_L1a[dict_index],
                                  dummy_val=int(ds_L1a[dict_index].Dummy.data))
        ds_ml[dict_index] = ss.oscar.level1.compute_multilooking_Master_Slave(
                                ds_L1a[dict_index],
                                window=7)
        ds_ml[dict_index]['Polarization'] = ss.oscar.level1.check_antenna_polarization(ds_L1a[dict_index])
        ds_ml[dict_index]['Baseline'] = ss.oscar.level1.compute_antenna_baseline(0.2)
        ds_ml[dict_index]['AntennaAzimuthImage'] =ss.oscar.level1.compute_antenna_azimuth_direction(
                                                    ds_L1a[dict_index],
                                                    antenna=antenna_ident[ds_L1a_dict_index.index(dict_index)])
        ds_ml[dict_index]['TimeLag'] = ss.oscar.level1.compute_time_lag_Master_Slave(
                                           ds_L1a[dict_index],
                                           options='from_SAR_time')
        ds_ml[dict_index]['RadialSurfaceVelocity'] = ss.oscar.level1.compute_radial_surface_velocity(
                                                         ds_L1a[dict_index],
                                                         ds_ml[dict_index])

        ds_ml[dict_index][vars_to_keep] = ds_L1a[dict_index][vars_to_keep]
        ds_ml[dict_index].attrs['Title'] = ds_L1a[dict_index].Title
        ds_ml[dict_index]['TrackTime'] = track_title_to_datetime(ds_ml[dict_index].Title)
#-----------------------------------------------------------
    ds_ml = ss.oscar.level1.fill_missing_variables(ds_ml, antenna_ident)

# Build L1b dataset
#aft_id = list(ds_ml.keys())[antenna_ident.index('Aft')]
#ds_ml[aft_id]['CrossRange'] = ds_ml[aft_id].CrossRange + 2
    ds_L1b[track] = ss.oscar.level1.merge_beams(ds_ml, antenna_ident)
    del ds_ml


Device name = LIVPCSW-10
Setting local paths...
OSCAR L1 processing - version v20221104
-------------------
File path : D:\data\oscar\v20221104\matlab-pre-processed\
-------------------
File index : 0 , Aquisition time : 20220522T052218 , File list indices : [0, 1, 2]
File index : 1 , Aquisition time : 20220522T053942 , File list indices : [3, 4, 5]
File index : 2 , Aquisition time : 20220522T054805 , File list indices : [6, 7, 8]
File index : 3 , Aquisition time : 20220522T055458 , File list indices : [9, 10, 11]
File index : 4 , Aquisition time : 20220522T062018 , File list indices : [12, 13, 14]
File index : 5 , Aquisition time : 20220522T062620 , File list indices : [15, 16, 17]
File index : 6 , Aquisition time : 20220522T063239 , File list indices : [18, 19, 20]
File index : 7 , Aquisition time : 20220522T063906 , File list indices : [21, 22, 23]
File index : 8 , Aquisition time : 20220522T064459 , File list indices : [24, 25, 26]
File index : 9 , Aquisition time : 20220522T065108

In [10]:
folder = 'restricted'
data_type = 'METEOFRANCE_AROME'
AROME_data_folder = os.path.join(file_paths['aux_path'], folder, data_type,'')

AROME_dict = dict()
AROME_file_list = os.listdir(AROME_data_folder)
for file in AROME_file_list:
    AROME_dict[AROME_file_list.index(file)] =\
    xr.open_dataset(AROME_data_folder + file)

AROME_winds = xr.concat(list(AROME_dict.values()),
                        dim='time',
                       join='outer')
del AROME_dict


In [11]:
AROME_winds

<xarray.Dataset>
Dimensions:    (time: 14, latitude: 281, longitude: 321, height: 1)
Coordinates:
  * time       (time) datetime64[ns] 2022-05-17T07:00:00 ... 2022-05-26T18:00:00
  * latitude   (latitude) float32 43.0 43.03 43.05 43.08 ... 49.95 49.97 50.0
  * longitude  (longitude) float32 -8.0 -7.975 -7.95 -7.925 ... -0.05 -0.025 0.0
  * height     (height) float32 0.0
Data variables:
    hu2m       (time, height, latitude, longitude) float32 0.915 ... 0.8615
    nebul      (time, height, latitude, longitude) float32 1.0 1.0 ... 0.93
    pmer       (time, height, latitude, longitude) float32 1.018e+05 ... 1.02...
    t2m        (time, height, latitude, longitude) float32 287.2 287.3 ... 286.8
    u10m       (time, height, latitude, longitude) float32 -0.08508 ... 10.45
    v10m       (time, height, latitude, longitude) float32 6.063 6.329 ... 2.802
Attributes: (12/40)
    data_type:               OCO straight grid
    format_version:          1.2
    title:                   Meteo-France Arome analysis and forecast
    Conventions:             CF-1.3
    netcdf_version:          3.5
    product_name:            METEOFRANCE_AROME_20220517T07Z.nc
    ...                      ...
    data_centre:             CD-OCO
    data_centre_references:  http://www.previmer.org/
    contact:                 cdoco-exploit@ifremer.fr
    distribution_statement:  Data restrictions: for registered users only
    operational_status:      operational
    quality_index:           1

In [21]:
ds_L1b['Track_13'].latitude.median()
u10 = AROME_winds.u10m.interp(time=ds_L1b['Track_13'].sel(Antenna='Fore').TrackTime,method='nearest')\
.interp(latitude=ds_L1b['Track_13'].latitude.median(), method='nearest')\
.interp(longitude=ds_L1b['Track_13'].longitude.median(), method='nearest')
v10 = AROME_winds.v10m.interp(time=ds_L1b['Track_13'].sel(Antenna='Fore').TrackTime,method='nearest')\
.interp(latitude=ds_L1b['Track_13'].latitude.median(), method='nearest')\
.interp(longitude=ds_L1b['Track_13'].longitude.median(), method='nearest')

C:\Users\dmca\Anaconda3\envs\seastar\lib\site-packages\xarray\core\missing.py:562: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
C:\Users\dmca\Anaconda3\envs\seastar\lib\site-packages\xarray\core\missing.py:563: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
C:\Users\dmca\Anaconda3\envs\seastar\lib\site-packages\xarray\core\missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
C:\Users\dmca\Anaconda3\envs\seastar\lib\site-packages\xarray\core\missing.py:563: FutureWarning: Passing method to Float64Index.get_loc 

<xarray.DataArray 'u10m' (height: 1)>
array([-3.816474], dtype=float32)
Coordinates:
  * height     (height) float32 0.0
    time       datetime64[ns] 2022-05-22T06:32:39
    Antenna    <U4 'Fore'
    latitude   float64 48.23
    longitude  float64 -5.278
Attributes:
    long_name:      Composante U du vent horizontal
    standard_name:  eastward_wind
    units:          m s-1
    valid_min:      -100.0
    valid_max:      100.0

In [24]:
u10 = AROME_winds.u10m.interp(time=ds_L1b['Track_13'].sel(Antenna='Fore').TrackTime,method='nearest')\
    .interp(latitude=ds_L1b['Track_13'].latitude.median(), method='nearest')\
    .interp(longitude=ds_L1b['Track_13'].longitude.median(), method='nearest')
v10 = AROME_winds.v10m.interp(time=ds_L1b['Track_13'].sel(Antenna='Fore').TrackTime,method='nearest')\
    .interp(latitude=ds_L1b['Track_13'].latitude.median(), method='nearest')\
    .interp(longitude=ds_L1b['Track_13'].longitude.median(), method='nearest')
windspeed, winddir = ss.utils.tools.windUV2SpeedDir(u10, v10)

print(windspeed.data)
print(winddir)


[5.6119537]
[42.84845]


C:\Users\dmca\Anaconda3\envs\seastar\lib\site-packages\xarray\core\missing.py:562: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
C:\Users\dmca\Anaconda3\envs\seastar\lib\site-packages\xarray\core\missing.py:563: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
C:\Users\dmca\Anaconda3\envs\seastar\lib\site-packages\xarray\core\missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
C:\Users\dmca\Anaconda3\envs\seastar\lib\site-packages\xarray\core\missing.py:563: FutureWarning: Passing method to Float64Index.get_loc 

In [ ]:
ds_L1b['Track_13'].sel(Antenna='Fore').TrackTime

In [ ]:
AROME_winds.time - ds_L1b['Track_13'].sel(Antenna='Fore').TrackTime

In [4]:
def track_title_to_datetime(title):

    year = title.split()[2].split('T')[0][0:4]
    month = title.split()[2].split('T')[0][4:6]
    day = title.split()[2].split('T')[0][6:8]
    hour = title.split()[2].split('T')[1][0:2]
    minute = title.split()[2].split('T')[1][2:4]
    second = title.split()[2].split('T')[1][4:6]
    track_time = np.datetime64(year + '-' + month + '-' + day + 'T' + hour + ':' + minute + ':' + second)
    return track_time

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1,1,1, projection=ccrs.Mercator())
np.sqrt(
    Arome_1.isel(time=0).u10m ** 2
    + Arome_1.isel(time=0).v10m ** 2)\
 .plot(
    y='latitude',
    x='longitude',
    robust=True, 
    cmap='coolwarm',
    transform=ccrs.PlateCarree()
)

gl=ax.gridlines(draw_labels=True)
ax.set_extent([-5.2, -5.0, 48.37, 48.5], crs=ccrs.PlateCarree())
ax.add_feature(cfeature.GSHHSFeature(scale='full',facecolor='white'))

In [ ]:
ds_L1b.keys()

In [ ]:
ds_L1a.keys()
for beam in ['Fore', 'Mid', 'Aft']:
    ax1 = ds_L1b[track].Intensity.sel(Antenna=beam).plot(figsize=(30, 6),
                        y='GroundRange', x='CrossRange',
                        robust=True,
                        cmap='twilight',

                      )

In [3]:
def merge_beams(ds_dict, antenna_id):

    ds_level1 = xr.concat(objs=list(ds_dict.values()),
                          dim='Antenna',
                          join='outer',
                          coords='all',
                         )
    
    ds_level1 = ds_level1.assign_coords(Antenna=('Antenna', antenna_id))
    key_list = list(ds_dict.keys())
    ds_level1.coords['latitude'] = xr.merge(
            [ds_dict[key_list[0]].LatImage.dropna(dim='CrossRange'),
             ds_dict[key_list[1]].LatImage.dropna(dim='CrossRange'),
             ds_dict[key_list[2]].LatImage.dropna(dim='CrossRange')],
            ).LatImage
    ds_level1.coords['longitude'] = xr.merge(
            [ds_dict[key_list[0]].LonImage.dropna(dim='CrossRange'),
             ds_dict[key_list[1]].LonImage.dropna(dim='CrossRange'),
             ds_dict[key_list[2]].LonImage.dropna(dim='CrossRange')],
            ).LonImage

    return ds_level1